In [79]:
import math
import numpy as np
import pandas as pd
from setuptools.wheel import unpack
from ydata_profiling import ProfileReport

df = pd.read_csv("./dataset/requests.csv")
df = df.rename(columns={'timestamp [UTC]': 'timestamp', "name": "request_line", "resultCode": "response_code",
                        "duration": "duration_ms"})
df = df[['timestamp', 'request_line', 'response_code', 'duration_ms']]

In [69]:
# query all request having response_code != 200
error_requests = df[df['response_code'] != 200]
error_requests

,timestamp,request_line,response_code,duration_ms
320,"9/23/2025, 11:10:13.883 AM",GET /robots933456.txt,404,27
359,"9/23/2025, 11:14:44.552 AM",POST /bot/api/messages,415,3
527,"9/24/2025, 1:29:32.713 AM",POST /chat_with_data_v2/stream,500,10167
652,"9/24/2025, 1:42:51.844 AM",POST /chat_with_data_v2/stream,500,8898
1991,"9/24/2025, 12:11:30.722 AM",POST /chat_with_data_v2/stream,500,13045
2843,"9/23/2025, 10:15:46.499 AM",GET /robots933456.txt,404,27
2848,"9/23/2025, 10:15:49.706 AM",GET /robots933456.txt,404,27
3507,"9/23/2025, 1:40:54.282 PM",POST /chat_with_data_v2/stream,500,8046
4030,"9/24/2025, 12:07:05.216 AM",POST /chat_with_data_v2/stream,500,610499
4573,"9/24/2025, 1:04:28.410 AM",POST /chat_with_data_v2/stream,500,6240


In [88]:
# aggregate data frame group by request_line
agg_df = df.groupby('request_line').agg(
    total_requests=('request_line', 'count'),
    total_seconds=('duration_ms', lambda x: math.ceil(np.sum(x) / 1000)),
    mean_ms=('duration_ms', lambda x: math.ceil(np.mean(x))),
    median_ms=('duration_ms', lambda x: math.ceil(np.median(x))),
    max_ms=('duration_ms', 'max'),
    min_ms=('duration_ms', 'min'),
    errors=('response_code', lambda x: (x != 200).sum())
).reset_index()
agg_df = agg_df.sort_values(by='total_requests', ascending=False)
agg_df

,request_line,total_requests,total_seconds,mean_ms,median_ms,max_ms,min_ms,errors
16,POST /normal_v2/stream,5023,57653,11478,6496,680610,1328,0
2,GET /chat_with_data/{job_id},2790,45,16,12,354,7,0
0,GET /,2297,12,5,4,99,2,0
11,POST /chat_with_data_v2/stream,476,14832,31160,23002,610499,4344,7
10,POST /chat_with_data/file_list,206,82,394,249,4740,3,0
4,GET /mode/smes_modes,119,4,28,14,197,8,0
5,GET /multimodal/file_types/,81,1,4,3,29,2,0
1,GET /chat_with_data/file_types/,73,1,4,3,30,2,0
17,POST /web_search/stream,72,757,10507,9160,45719,4088,0
12,POST /deep_search/stream,50,5343,106843,50773,1834960,12714,0
